In [3]:
import pandas as pd

In [4]:
YEAR = 2023
TICKER = "SPX"

In [32]:
path = f"../modules/stochastic_volatility_models/stochastic_volatility_models/data/wrds/options/{YEAR}/option_prices_{TICKER.lower()}_{YEAR}.csv"
option_prices = pd.read_csv(path).drop(columns="Unnamed: 0")

In [72]:
option_prices = option_prices.set_index(["cp_flag", "exdate", "strike_price", "am_settlement", "date"])

---

In [62]:
# option_prices = option_prices.drop(columns=["forward_price", "root", "suffix"])


In [73]:
option_prices.head()

symbol  \
cp_flag exdate     strike_price am_settlement date                             
C       2023-01-20 1000000.0    1.0           2023-01-03  SPX 230120C1000000   
                   1200000.0    1.0           2023-01-03  SPX 230120C1200000   
                   1400000.0    1.0           2023-01-03  SPX 230120C1400000   
                   1500000.0    1.0           2023-01-03  SPX 230120C1500000   
                   1600000.0    1.0           2023-01-03  SPX 230120C1600000   

                                                           last_date  \
cp_flag exdate     strike_price am_settlement date                     
C       2023-01-20 1000000.0    1.0           2023-01-03  2023-01-03   
                   1200000.0    1.0           2023-01-03  2022-12-21   
                   1400000.0    1.0           2023-01-03  2022-12-21   
                   1500000.0    1.0           2023-01-03  2022-12-29   
                   1600000.0    1.0           2023-01-03  2022-11-14   

                                                          best_bid  \
cp_flag exdate     strike_price am_settlement date                   
C       2023-01-20 1000000.0    1.0           2023-01-03    2823.2   
                   1200000.0    1.0           2023-01-03    2623.6   
                   1400000.0    1.0           2023-01-03    2424.1   
                   1500000.0    1.0           2023-01-03    2324.4   
                   1600000.0    1.0           2023-01-03    2224.6   

                                                          best_offer  volume  \
cp_flag exdate     strike_price am_settlement date                             
C       2023-01-20 1000000.0    1.0           2023-01-03      2826.6     2.0   
                   1200000.0    1.0           2023-01-03      2627.1     0.0   
                   1400000.0    1.0           2023-01-03      2427.5     0.0   
                   1500000.0    1.0           2023-01-03      2328.0     0.0   
                   1600000.0    1.0           2023-01-03      2228.0     0.0   

                                                          open_interest  \
cp_flag exdate     strike_price am_settlement date                        
C       2023-01-20 1000000.0    1.0           2023-01-03         3557.0   
                   1200000.0    1.0           2023-01-03          149.0   
                   1400000.0    1.0           2023-01-03          262.0   
                   1500000.0    1.0           2023-01-03           10.0   
                   1600000.0    1.0           2023-01-03           56.0   

                                                          impl_volatility  \
cp_flag exdate     strike_price am_settlement date                          
C       2023-01-20 1000000.0    1.0           2023-01-03              NaN   
                   1200000.0    1.0           2023-01-03              NaN   
                   1400000.0    1.0           2023-01-03              NaN   
                   1500000.0    1.0           2023-01-03              NaN   
                   1600000.0    1.0           2023-01-03              NaN   

                                                          delta  gamma  vega  \
cp_flag exdate     strike_price am_settlement date                             
C       2023-01-20 1000000.0    1.0           2023-01-03    NaN    NaN   NaN   
                   1200000.0    1.0           2023-01-03    NaN    NaN   NaN   
                   1400000.0    1.0           2023-01-03    NaN    NaN   NaN   
                   1500000.0    1.0           2023-01-03    NaN    NaN   NaN   
                   1600000.0    1.0           2023-01-03    NaN    NaN   NaN   

                                                          theta     optionid  \
cp_flag exdate     strike_price am_settlement date                             
C       2023-01-20 1000000.0    1.0           2023-01-03    NaN  140692651.0   
                   1200000.0    1.0           2023-01-03    NaN  140692652.0   
     

In [97]:
filtered_groups = option_prices.xs(1.0, level=3).groupby(level=[0, 1, 2]).first()

In [102]:
print(filtered_groups.index.shape)
for i in range(3):
	print(filtered_groups.index.get_level_values(i).unique().shape)

(11724,)
(2,)
(27,)
(466,)


In [105]:
filtered_groups

symbol   last_date  best_bid  \
cp_flag exdate     strike_price                                              
C       2023-01-20 200000.0        SPX 230120C200000  2022-12-30    3621.2   
                   400000.0        SPX 230120C400000  2022-12-12    3421.6   
                   600000.0        SPX 230120C600000  2022-12-16    3222.1   
                   800000.0        SPX 230120C800000  2022-11-23    3022.7   
                   1000000.0      SPX 230120C1000000  2023-01-03    2823.2   
...                                              ...         ...       ...   
P       2028-12-15 8800000.0      SPX 281215P8800000        None    2938.1   
                   9200000.0      SPX 281215P9200000        None    3251.0   
                   9600000.0      SPX 281215P9600000  2023-06-20    3566.7   
                   10000000.0    SPX 281215P10000000        None    3856.5   
                   12000000.0    SPX 281215P12000000  2023-06-27    5453.4   

                                 best_offer  volume  open_interest  \
cp_flag exdate     strike_price                                      
C       2023-01-20 200000.0          3624.9     0.0         1125.0   
                   400000.0          3425.4     0.0          703.0   
                   600000.0          3226.1     0.0           79.0   
                   800000.0          3026.2     0.0           64.0   
                   1000000.0         2826.6     2.0         3557.0   
...                                     ...     ...            ...   
P       2028-12-15 8800000.0         3130.1     0.0            0.0   
                   9200000.0         3443.0     0.0            0.0   
                   9600000.0         3758.7     2.0            0.0   
                   10000000.0        4048.5     0.0            0.0   
                   12000000.0        5645.4     3.0            0.0   

                                 impl_volatility     delta     gamma  \
cp_flag exdate     strike_price                                        
C       2023-01-20 200000.0             7.593877  0.999818  0.000000   
                   400000.0             3.714910  0.999969  0.000000   
                   600000.0             3.405508  0.999841  0.000000   
                   800000.0             3.078620  0.999613  0.000001   
                   1000000.0            2.678704  0.999498  0.000001   
...                                          ...       ...       ...   
P       2028-12-15 8800000.0            0.186620 -0.756743  0.000103   
                   9200000.0            0.198105 -0.758241  0.000097   
                   9600000.0            0.210444 -0.757263  0.000091   
                   10000000.0           0.231817 -0.744818  0.000092   
                   12000000.0           0.295591 -0.726133  0.000078   

                                        vega      theta     optionid  \
cp_flag exdate     strike_price                                        
C       2023-01-20 200000.0         0.356750  -90.39736  140692656.0   
                   400000.0         0.083018  -21.71299  140692686.0   
                   600000.0         0.392986  -48.53062  140692715.0   
                   800000.0         0.903667  -82.88274  140692723.0   
                   1000000.0        1.150166  -96.36358  140692651.0   
...                                      ...        ...          ...   
P       2028-12-15 8800000.0     2041.111000  187.43570  155270306.0   
                   9200000.0     2024.563000  200.42570  155270307.0   
                   9600000.0     2035.377000  212.63980  155270308.0   
                   10000000.0    2228.027000  227.10000  155380205.0   
                   12000000.0    2409.081000  283.42420  155380206.0   

                                expiry_indicator  
cp_flag exdate     strike_price                   
C       2023-01-20 200000.0                 None  
                   400000.0                 None  
                   600000.0               

In [104]:
11724 / 2

217.11111111111111

In [95]:
filtered_groups.shape

(12, 13)

In [93]:
994262 / 2 / 167

2976.8323353293413

In [89]:
option_prices.xs(1.0, level=3).index.get_level_values(3).unique().shape
option_prices.xs(1.0, level=3).shape

(994262, 13)

In [75]:
option_prices.xs(1.0, level=3).shape

(994262, 13)

In [71]:
option_prices[(option_prices["exdate"] == "2023-01-20") & (option_prices["strike_price"] == 200000.0) & (option_prices["cp_flag"] == "C") & (option_prices["date"] == "2023-01-10") & (option_prices["am_settlement"] == 1)]

,date,symbol,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,am_settlement,expiry_indicator
89472,2023-01-10,SPX 230120C200000,2023-01-20,2023-01-05,C,200000.0,3715.6,3720.3,0.0,1125.0,NaN,NaN,NaN,NaN,NaN,140692656.0,1.0,NaN


In [69]:
option_prices["strike_price"].unique()

array([ 1000000.,  1200000.,  1400000.,  1500000.,  1600000.,  1700000.,
        1800000.,  1900000.,   200000.,  2000000.,  2100000.,  2200000.,
        2300000.,  2350000.,  2400000.,  2450000.,  2500000.,  2550000.,
        2600000.,  2625000.,  2650000.,  2675000.,  2700000.,  2725000.,
        2750000.,  2775000.,  2800000.,  2825000.,  2850000.,  2875000.,
        2900000.,  2925000.,  2950000.,  2975000.,  3000000.,  3025000.,
        3050000.,  3060000.,  3070000.,  3075000.,  3080000.,  3090000.,
        3100000.,  3110000.,  3120000.,  3125000.,  3130000.,  3140000.,
        3150000.,  3160000.,  3170000.,  3175000.,  4470000.,  3180000.,
        3190000.,  3200000.,  3210000.,  3220000.,  3225000.,  3230000.,
        3240000.,  3250000.,  3260000.,  3270000.,  3275000.,  3280000.,
        3290000.,  3300000.,  3310000.,  3320000.,  3325000.,  3330000.,
        3340000.,  3350000.,  3360000.,  3365000.,  3370000.,  3375000.,
        3380000.,  3385000.,  3390000.,  3395000., 

In [48]:
option_prices[option_prices["symbol"] == "SPX 230120C1000000"]

,date,symbol,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,...,delta,gamma,vega,theta,optionid,am_settlement,forward_price,expiry_indicator,root,suffix
0,2023-01-03,SPX 230120C1000000,2023-01-20,2023-01-03,C,1000000.0,2823.2,2826.6,2.0,3557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN
17733,2023-01-04,SPX 230120C1000000,2023-01-20,2023-01-04,C,1000000.0,2847.5,2850.5,4.0,3557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN
35706,2023-01-05,SPX 230120C1000000,2023-01-20,2023-01-04,C,1000000.0,2806.9,2809.2,0.0,3557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN
53706,2023-01-06,SPX 230120C1000000,2023-01-20,2023-01-06,C,1000000.0,2891.8,2894.6,2000.0,3557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN
71732,2023-01-09,SPX 230120C1000000,2023-01-20,2023-01-06,C,1000000.0,2891.2,2895.7,0.0,1557.0,...,0.999498,0.000001,1.150166,-96.36358,140692651.0,1.0,NaN,NaN,NaN,NaN
89464,2023-01-10,SPX 230120C1000000,2023-01-20,2023-01-06,C,1000000.0,2916.7,2921.4,0.0,1557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN
107292,2023-01-11,SPX 230120C1000000,2023-01-20,2023-01-06,C,1000000.0,2964.4,2969.1,0.0,1557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN
125225,2023-01-12,SPX 230120C1000000,2023-01-20,2023-01-06,C,1000000.0,2980.5,2984.8,0.0,1557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN
143476,2023-01-13,SPX 230120C1000000,2023-01-20,2023-01-06,C,1000000.0,2998.0,3002.1,0.0,1557.0,...,0.999453,0.000001,0.990734,-147.40040,140692651.0,1.0,NaN,NaN,NaN,NaN
161714,2023-01-17,SPX 230120C1000000,2023-01-20,2023-01-06,C,1000000.0,2988.2,2991.9,0.0,1557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN


In [40]:
option_prices[option_prices["date"] == "2023-01-03"].drop("columns")

,date,symbol,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,...,delta,gamma,vega,theta,optionid,am_settlement,forward_price,expiry_indicator,root,suffix
0,2023-01-03,SPX 230120C1000000,2023-01-20,2023-01-03,C,1000000.0,2823.20,2826.6,2.0,3557.0,...,NaN,NaN,NaN,NaN,140692651.0,1.0,NaN,NaN,NaN,NaN
1,2023-01-03,SPX 230120C1200000,2023-01-20,2022-12-21,C,1200000.0,2623.60,2627.1,0.0,149.0,...,NaN,NaN,NaN,NaN,140692652.0,1.0,NaN,NaN,NaN,NaN
2,2023-01-03,SPX 230120C1400000,2023-01-20,2022-12-21,C,1400000.0,2424.10,2427.5,0.0,262.0,...,NaN,NaN,NaN,NaN,140692653.0,1.0,NaN,NaN,NaN,NaN
3,2023-01-03,SPX 230120C1500000,2023-01-20,2022-12-29,C,1500000.0,2324.40,2328.0,0.0,10.0,...,NaN,NaN,NaN,NaN,149952041.0,1.0,NaN,NaN,NaN,NaN
4,2023-01-03,SPX 230120C1600000,2023-01-20,2022-11-14,C,1600000.0,2224.60,2228.0,0.0,56.0,...,NaN,NaN,NaN,NaN,140692654.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17728,2023-01-03,SPXW 231229P5000000,2023-12-29,2022-12-29,P,5000000.0,1010.10,1025.5,0.0,463.0,...,-0.891393,0.000259,628.6868,116.263500,151576480.0,0.0,NaN,m,NaN,NaN
17729,2023-01-03,SPXW 231229P5025000,2023-12-29,NaN,P,5025000.0,1032.10,1047.5,0.0,0.0,...,-0.896657,0.000248,602.0881,120.441300,151647377.0,0.0,NaN,m,NaN,NaN
17730,2023-01-03,SPXW 231229P5200000,2023-12-29,2022-12-29,P,5200000.0,1189.00,1205.1,0.0,3.0,...,-0.925785,0.000183,441.9281,146.113600,151576481.0,0.0,NaN,m,NaN,NaN
17731,2023-01-03,SPXW 231229P5400000,2023-12-29,NaN,P,5400000.0,1373.20,1389.8,0.0,0.0,...,-0.946253,0.000128,313.0978,168.854500,151576482.0,0.0,NaN,m,NaN,NaN


In [26]:
from datetime import datetime

ts = option_prices["date"].apply(lambda date: datetime.strptime(date, "%Y-%m-%d"))

In [27]:
ts

0         2023-01-03
1         2023-01-03
2         2023-01-03
3         2023-01-03
4         2023-01-03
             ...    
3086603   2023-08-31
3086604   2023-08-31
3086605   2023-08-31
3086606   2023-08-31
3086607   2023-08-31
Name: date, Length: 3086608, dtype: datetime64[ns]